In [5]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# ==============================================================
# 1. Load Data & Define Standardization Functions
# ==============================================================
df = pd.read_excel('C:/Users/user/OneDrive/gym_recommendation_system/venv/data/gym recommendation.xlsx')

def standardize_text(text):
    """Clean and normalize text entries"""
    text = re.sub(r'[^a-zA-Z0-9 ]', '', str(text))  # Remove special chars
    text = text.strip().lower()
    # Manual term replacements
    replacements = {
        'dumbbells': 'dumbbell',
        'treadmills': 'treadmill',
        'swimming.': 'swimming',
        'carrots': 'carrot',
        'male': 0, 'female': 1,  # Predefine gender encoding
        'yes': 1, 'no': 0        # Predefine binary flags
    }
    return replacements.get(text, text)

# =================================================================
# 2. Process Multi-Label Columns
# =================================================================
def process_diet(diet_str):
    """Process diet column with nested structure"""
    diet_str = re.sub(r'\w+:\s*', '', str(diet_str)) 
    items = re.findall(r'\((.*?)\)', diet_str)
    return [standardize_text(item) 
            for group in items 
            for item in re.split(r', | and ', group) 
            if item.strip()]

def process_generic(column_str):
    """Process Exercises/Equipment columns"""
    return [standardize_text(item) 
            for item in re.split(r', | and ', str(column_str)) 
            if item.strip()]

# Apply processing
df['Diet'] = df['Diet'].apply(process_diet)
df['Exercises'] = df['Exercises'].apply(process_generic)
df['Equipment'] = df['Equipment'].apply(process_generic)

# =================================================================
# 3. Encode Multi-Label Features
# =================================================================
class DedupeMLB(MultiLabelBinarizer):
    """Custom MLB with text standardization"""
    def fit_transform(self, y):
        y_clean = [[standardize_text(item) for item in lst] for lst in y]
        return super().fit_transform(y_clean)

def safe_encode(df, column):
    """Encoding with automatic deduplication"""
    mlb = DedupeMLB()
    encoded = pd.DataFrame(
        mlb.fit_transform(df[column]),
        columns=[f"{column}_{item.replace(' ', '_')}" for item in mlb.classes_],
        index=df.index
    )
    return encoded.T.groupby(level=0).max().T

# Encode all multi-label columns
diet_encoded = safe_encode(df, 'Diet')
exercises_encoded = safe_encode(df, 'Exercises')
equipment_encoded = safe_encode(df, 'Equipment')

# =================================================================
# 4. Encode Categorical Variables
# =================================================================
# Convert binary columns
binary_cols = ['Sex', 'Hypertension', 'Diabetes']
df[binary_cols] = df[binary_cols].applymap(standardize_text)

# One-hot encode multi-class columns
cat_cols = ['Level', 'Fitness Goal', 'Fitness Type']
ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe_features = ohe.fit_transform(df[cat_cols])
ohe_df = pd.DataFrame(ohe_features, columns=ohe.get_feature_names_out(cat_cols))

# =================================================================
# 5. Create Final DataFrame
# =================================================================
final_df = pd.concat([
    df.drop(['Diet', 'Exercises', 'Equipment', 'ID', 'Recommendation'] + cat_cols, axis=1),
    ohe_df,
    diet_encoded.add_prefix('diet_'),
    exercises_encoded.add_prefix('exercise_'),
    equipment_encoded.add_prefix('equipment_')
], axis=1).astype(float)

# =================================================================
# 6. Model Training (Revised)
# =================================================================
# Verify columns
print("Final features:", final_df.columns.tolist()[:30], "...")

# Isolate key features
feature_cols = ['Age', 'BMI', 'Hypertension', 'Fitness Goal_Weight Loss']
target_col = 'diet_Diet_and_lowfat_dairy_products'

X = final_df[feature_cols]
y = final_df[target_col]

# Retrain model
model = RandomForestClassifier()
model.fit(X, y)

# Show interpretable features
importance = pd.Series(model.feature_importances_, index=feature_cols)
print("\nKey Drivers of Diet Choices:")
print(importance.sort_values(ascending=False))

# =================================================================
# 7. Output Sample
# =================================================================
print("\nSample preprocessed data:")
print(final_df.iloc[:2, :5].T)

C:\Users\user\AppData\Local\Temp\ipykernel_40916\2478514432.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[binary_cols] = df[binary_cols].applymap(standardize_text)


Final features: ['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'BMI', 'Level_Obuse', 'Level_Overweight', 'Level_Underweight', 'Fitness Goal_Weight Loss', 'Fitness Type_Muscular Fitness', 'diet_Diet_aloe_vera_juice', 'diet_Diet_and_apple', 'diet_Diet_and_baru_nuts', 'diet_Diet_and_beetroot_juice', 'diet_Diet_and_bell_peppers', 'diet_Diet_and_black_walnut', 'diet_Diet_and_hemp_seeds', 'diet_Diet_and_iceberg_lettuce', 'diet_Diet_and_jelly_sandwich', 'diet_Diet_and_lettuce', 'diet_Diet_and_lowfat_dairy_products', 'diet_Diet_and_mango_juice', 'diet_Diet_and_mixed_teff', 'diet_Diet_and_nuts', 'diet_Diet_and_watermelon_juice', 'diet_Diet_apple_juice', 'diet_Diet_baru_nuts', 'diet_Diet_beech_nuts'] ...

Key Drivers of Diet Choices:
BMI                         0.535659
Hypertension                0.290536
Age                         0.118505
Fitness Goal_Weight Loss    0.055299
dtype: float64

Sample preprocessed data:
                  0      1
Sex            0.00   0.00
Age   

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Helper function to compute metrics
def compute_metrics(true_labels, scores, threshold=0.5):
    """Compute precision, recall, F1, and AUC for given labels and scores."""
    pred_labels = (scores > threshold).astype(int)
    precision = precision_score(true_labels, pred_labels, zero_division=0)
    recall = recall_score(true_labels, pred_labels, zero_division=0)
    f1 = f1_score(true_labels, pred_labels, zero_division=0)
    auc = roc_auc_score(true_labels, scores) if len(np.unique(true_labels)) > 1 else 0
    return precision, recall, f1, auc

# Define your target column (example)
target_col = 'diet_Diet_and_lowfat_dairy_products'  # Adjust based on your dataset

# Step 1: Identify diet plan columns
plan_cols = [col for col in final_df.columns if col.startswith('diet_')]

# Step 2: Create a unique identifier for each diet plan
# Convert binary vectors to tuples for grouping
final_df['plan_id'] = final_df[plan_cols].apply(lambda row: tuple(row), axis=1)

# Step 3: Get unique diet plans
unique_plans = final_df['plan_id'].unique()

# Step 4: Split unique plans into training and validation sets
train_plans, val_plans = train_test_split(unique_plans, test_size=0.2, random_state=42)

# Step 5: Assign rows to training and validation based on their plan_id
train_df = final_df[final_df['plan_id'].isin(train_plans)]
val_df = final_df[final_df['plan_id'].isin(val_plans)]

# Step 6: Ensure no overlap between training and validation plans
assert set(train_df['plan_id']) & set(val_df['plan_id']) == set(), "Overlap detected between training and validation plans!"

# Step 7: Extract plan features and labels
train_plan = train_df[plan_cols].values
train_labels = train_df[target_col].values
val_plan = val_df[plan_cols].values
val_labels = val_df[target_col].values

# Step 8: Popularity Baseline
# Extract positive plans from the training set
train_plan_pos = train_plan[train_labels == 1]

# Compute unique plans and their frequencies
unique_plans_train, counts = np.unique(train_plan_pos, axis=0, return_counts=True)
plan_freq = counts / counts.sum()  # Normalize frequencies

# Create a dictionary mapping each unique plan (as a tuple) to its frequency
plan_to_freq = {tuple(plan): freq for plan, freq in zip(unique_plans_train, plan_freq)}

# Convert validation plans to tuples and assign frequency scores
val_plan_tuples = [tuple(plan) for plan in val_plan]
val_scores_pop = np.array([plan_to_freq.get(plan, 0) for plan in val_plan_tuples])

# Step 9: Compute metrics for the popularity baseline
precision_pop, recall_pop, f1_pop, auc_pop = compute_metrics(val_labels, val_scores_pop)
print("Popularity Baseline Metrics:")
print(f"Precision: {precision_pop:.4f}, Recall: {recall_pop:.4f}, F1: {f1_pop:.4f}, AUC: {auc_pop:.4f}")

# Optional: Check if any validation plans are in the training positive plans
overlap_count = sum(1 for plan in val_plan_tuples if plan in plan_to_freq)
print(f"Number of validation plans found in training positive plans: {overlap_count}")

Popularity Baseline Metrics:
Precision: 0.0000, Recall: 0.0000, F1: 0.0000, AUC: 0.0000
Number of validation plans found in training positive plans: 0
